## Máster en Big Data y Data Science

### Metodologías de gestión y diseño de proyectos de big data

#### AP2 - Modelado y evaluación

---

En esta libreta se realiza la experimentación para generación del modelo de predicción objetivo del proyecto y la evaluación del mismo.
La versión del dataset a utilizar es la obtenida a partir de las operaciones de transformación.

---

En esta versión de la libreta se va a incorporta el registro de los detalles de la experimentación con la librería MLFlow

In [1]:
# Se importan las librerías necesarias y se suprimen las advertencias
import pandas as pd
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore',category=UserWarning)

Se agrega la librería mlflow y se configura inicialmente

In [17]:
import mlflow
import mlflow.sklearn
from datetime import datetime

# Configuración de MLFlow
mlflow.set_tracking_uri("file:../mlruns")
mlflow.set_experiment("Proyecto 13MBID-ABR2526 - Experimentación Original")

<Experiment: artifact_location=('file:///Users/juanjo/Documents/Maestría/13 Metodología de Gestión y '
 'Diseño de Big '
 'Data/RepoGithub/13MBID-ABR2526-Juan_Jose_Perez/notebooks/../mlruns/156006986107326267'), creation_time=1772289281220, experiment_id='156006986107326267', last_update_time=1772289281220, lifecycle_stage='active', name='Proyecto 13MBID-ABR2526 - Experimentación Original', tags={}>

In [3]:
# Lectura de los datos
df = pd.read_csv('../data/processed/bank-processed.csv')
df.head(5)

,age,job,marital,education,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,56,housemaid,married,basic.4y,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
1,57,services,married,high.school,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
2,37,services,married,high.school,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
3,40,admin.,married,basic.6y,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
4,56,services,married,high.school,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0


In [4]:
# Se divide el dataset en variables predictoras y variable objetivo
X = df.drop('y', axis=1)
y = df['y']

In [5]:
# Se genera el conjunto de entrenamiento y prueba con estratificación
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

In [6]:
# Se separan las columnas numéricas
numerical_columns=X_train.select_dtypes(exclude='object').columns
display(numerical_columns)

categorical_columns=X_train.select_dtypes(include='object').columns
display(categorical_columns)

Index(['age', 'duration', 'campaign', 'pdays', 'previous', 'emp_var_rate',
       'cons_price_idx', 'cons_conf_idx', 'euribor3m', 'nr_employed'],
      dtype='object')

Index(['job', 'marital', 'education', 'housing', 'loan', 'contact', 'month',
       'day_of_week', 'poutcome'],
      dtype='object')

In [7]:
# Se verifica la distribución de la variable objetivo en el conjunto de entrenamiento
y_train.value_counts()

y
0    27179
1     3406
Name: count, dtype: int64

In [8]:
# Se crea un pipeline para preprocesamiento de datos
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler  

# Pipeline para valores numéricos
num_pipeline = Pipeline(steps=[
    ('RobustScaler', RobustScaler())
])

# Pipeline para valores categóricos
cat_pipeline = Pipeline(steps=[
    ('OneHotEncoder', OneHotEncoder(drop='first',sparse_output=False))
])

# Se configuran los preprocesadores
preprocessor_full = ColumnTransformer([
    ('num_pipeline', num_pipeline, numerical_columns),
    ('cat_pipeline', cat_pipeline, categorical_columns)
]).set_output(transform='pandas')

In [9]:
preprocessor_train_valid = ColumnTransformer([
    ('num_pipeline', num_pipeline, numerical_columns),
    ('cat_pipeline', cat_pipeline, categorical_columns)
]).set_output(transform='pandas')

In [10]:
# Se ajusta y transforma el conjunto de entrenamiento y prueba
x_train_prep = preprocessor_full.fit_transform(X_train)
x_test_prep = preprocessor_full.transform(X_test)

In [11]:
# Se aplica submuestreo a los datos preprocesados
from sklearn.utils import resample

# Combinar los datos preprocesados con las etiquetas
train_data = x_train_prep.copy()
train_data['target'] = y_train.reset_index(drop=True)

# Separar por clase
class_0 = train_data[train_data['target'] == 0]
class_1 = train_data[train_data['target'] == 1]

# Encontrar la clase minoritaria
min_count = min(len(class_0), len(class_1))

# Submuestreo balanceado - tomar una muestra igual al tamaño de la clase minoritaria
class_0_balanced = resample(class_0, n_samples=min_count, random_state=42)
class_1_balanced = resample(class_1, n_samples=min_count, random_state=42)

# Combinar las clases balanceadas
balanced_data = pd.concat([class_0_balanced, class_1_balanced])

# Separar características y objetivo
x_train_resampled = balanced_data.drop('target', axis=1)
y_train_resampled = balanced_data['target']

print(f"Tamaño original: {len(x_train_prep)}")
print(f"Tamaño balanceado: {len(x_train_resampled)}")
print(f"Distribución balanceada: {y_train_resampled.value_counts()}")

Tamaño original: 30585
Tamaño balanceado: 5438
Distribución balanceada: target
0.0    2719
1.0    2719
Name: count, dtype: int64


ESTA APARTADO SE VA A CAMBIAR

In [26]:
from sklearn.model_selection import cross_val_score


# Se genera una función para realizar validación cruzada
def cross_val(model):
    scores = cross_val_score(model,x_train_resampled , y_train_resampled, cv=5, scoring='f1')
    print('cross validation f1 scores',scores*100)
    print('cross validation f1 mean',scores.mean()*100)
    print('cross validation f1 std',scores.std())
    print('-'*50)
    scores = cross_val_score(model,x_train_resampled , y_train_resampled, cv=5, scoring='recall')
    print('cross validation recall scores',scores*100)
    print('cross validation recall mean',scores.mean()*100)
    print('cross validation recall std',scores.std())

---
El cambio consuste en que la función directamente registre los detalle de la experimentación en mlflow

In [15]:
from multiprocessing.spawn import import_main_path
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score
from mlflow.models import infer_signature

def cross_val_mlflow(model, model_name, params=None):
    """
    Realiza la validación cruzada de un modelo y registra los resultados en MLFlow
    
    Args:
        model: Modelo a validar
        model_name: Nombre del modelo
        params: Parámetros del modelo
    """
    with mlflow.start_run(run_name=model_name):
        f1_scores = cross_val_score(model, x_train_resampled, y_train_resampled, cv=5, scoring='f1')
        f1_mean = f1_scores.mean()
        f1_std = f1_scores.std()
        
        recall_scores = cross_val_score(model, x_train_resampled, y_train_resampled, cv=5, scoring='recall')
        recall_mean = recall_scores.mean()
        recall_std = recall_scores.std()

        precision_scores = cross_val_score(model, x_train_resampled, y_train_resampled, cv=5, scoring='precision')
        precision_mean = precision_scores.mean()
        precision_std = precision_scores.std()

        accuracy_scores = cross_val_score(model, x_train_resampled, y_train_resampled, cv=5, scoring='accuracy')
        accuracy_mean = accuracy_scores.mean()
        accuracy_std = accuracy_scores.std()

        #Entrenar al modelo
        model.fit(x_train_resampled, y_train_resampled)

        # hacemos predicciones
        y_pred = model.predict(x_test_prep)

        # Obtenemos model signature
        signature = infer_signature(x_train_resampled, y_pred)

        test_f1 = f1_score(y_test, y_pred)
        test_recall = recall_score(y_test, y_pred)
        test_precision = precision_score(y_test, y_pred)
        test_accuracy = accuracy_score(y_test, y_pred)
        
        # Registramos los parametros y metricas en MLFlow
        if params:
            mlflow.log_params(params)
        else:
            mlflow.log_params(model.get_params())
        
        mlflow.log_params({
            "train_samples": len(x_train_resampled),
            "test_samples": len(x_test_prep),
            "balancing_method": "undersampling",
            "cv_folds": 5
        })

        # Registramos las metricas de la validación cruzada
        mlflow.log_metrics({
            "cv_f1_mean": f1_mean,
            "cv_f1_std": f1_std,
            "cv_recall_mean": recall_mean,
            "cv_recall_std": recall_std,
            "cv_precision_mean": precision_mean,
            "cv_precision_std": precision_std,
            "cv_accuracy_mean": accuracy_mean,
            "cv_accuracy_std": accuracy_std
        })

        mlflow.log_metrics({
            "test_f1": test_f1,
            "test_recall": test_recall,
            "test_precision": test_precision,
            "test_accuracy": test_accuracy
        })

        # Registramos el modelo
        mlflow.sklearn.log_model(
            model, 
            artifact_path="model",
            signature=signature
        )

        print(f"Modelo {model_name} registrado en MLFlow con el id de ejecución {mlflow.active_run().info.run_id}")

        return model, {
            "cv_f1_mean": f1_mean,
            "cv_recall_mean": recall_mean,
            "cv_precision_mean": precision_mean,
            "cv_accuracy_mean": accuracy_mean,
            "test_f1": test_f1,
            "test_recall": test_recall,
            "test_precision": test_precision,
            "test_accuracy": test_accuracy
        }


Se cambias estas celdas por una que hace todas las llamadas

In [ ]:
# Se aplica un modelo de regresión logística
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=1,penalty='l2',solver='liblinear',random_state=1,max_iter=100,tol=0.000000001)

# cross validation scores
cross_val(lr)

In [ ]:
# LinearSVC
from sklearn.svm import LinearSVC
svc = LinearSVC(max_iter=10000,tol=0.001)

# cross validation scores
cross_val(svc)

In [ ]:
# knclassifier
from sklearn.neighbors import KNeighborsClassifier
knc = KNeighborsClassifier(n_neighbors=7)

# cross validation scores
cross_val(knc)

In [ ]:
# decision tree classifier
from sklearn.tree import DecisionTreeClassifier
tree=DecisionTreeClassifier()

# cross validation scores
cross_val(tree)

In [ ]:
# decision tree plot
from sklearn.tree import plot_tree
tree.fit(x_train_prep, y_train)
plot_tree(tree, filled=True, rounded=True,max_depth=2,fontsize=10)
plt.show()

In [32]:
# Se obtiene la matriz de confusión para el modelo
from sklearn.metrics import confusion_matrix

y_pred = tree.predict(x_test_prep)
cm = confusion_matrix(y_test, y_pred)
print(cm)


[[6336  460]
 [ 386  465]]


In [ ]:
# Se visualiza la matriz de confusión
from sklearn.metrics import ConfusionMatrixDisplay
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['no', 'yes'])
disp.plot()

-----
Esta es la nueva celda que hace la invocación al proceso con mlflow

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC

# Resultados 
resultados = {}

# Método 1: Regresión 
lr = LogisticRegression(C=1,penalty='l2',solver='liblinear',random_state=1,max_iter=100,tol=0.000000001)
model_lr, resultados_lr = cross_val_mlflow(lr, "Logistic Regression")
resultados["Logistic Regression"] = resultados_lr

# Método 2: SVC
svc = LinearSVC(max_iter=10000,tol=0.001)
model_svc, resultados_svc = cross_val_mlflow(svc, "Support Vector Machine")
resultados["Support Vector Machine"] = resultados_svc


# Método 3: KNN
knc = KNeighborsClassifier(n_neighbors=7)
model_knc, resultados_knc = cross_val_mlflow(knc, "K-Nearest Neighbors")
resultados["K-Nearest Neighbors"] = resultados_knc

# Método 4: Árbol de decisión
tree = DecisionTreeClassifier()
model_tree, resultados_tree = cross_val_mlflow(tree, "Decision Tree Classifier")
resultados["Decision Tree Classifier"] = resultados_tree




2026/02/28 10:13:29 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Modelo Logistic Regression registrado en MLFlow con el id de ejecución 01dd288bbc5b4f96bc9551ce539fb519


2026/02/28 10:13:34 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Modelo Support Vector Machine registrado en MLFlow con el id de ejecución c42897b6fe254e6b9948ff1633451606


2026/02/28 10:13:37 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Modelo K-Nearest Neighbors registrado en MLFlow con el id de ejecución b552b1d053e34988abd77df5e4c4d5d1


2026/02/28 10:13:41 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Modelo Decision Tree Classifier registrado en MLFlow con el id de ejecución 38a6100520344ae79f171bd688a60dbe


Se realiza una comparación de los modelos para seleccionar el que va a ser utilizado posteriormente

In [19]:
df_comparacion = pd.DataFrame(resultados).T
df_comparacion = df_comparacion.round(4)
df_comparacion = df_comparacion.sort_values(by='test_f1', ascending=False)

print(df_comparacion)

print("\nEl mejor modelo basado en el conjunto de prueba es: ", df_comparacion.index[0])
print(f"Valor de F1 en test: {df_comparacion.loc[df_comparacion.index[0], 'test_f1']}")
print(f"Valor de recall en test: {df_comparacion.loc[df_comparacion.index[0], 'test_recall']}")



                          cv_f1_mean  cv_recall_mean  cv_precision_mean  \
Decision Tree Classifier      0.6958          0.7381             0.6589   
K-Nearest Neighbors           0.5699          0.5815             0.5591   
Logistic Regression           0.5244          0.5204             0.5289   
Support Vector Machine        0.5241          0.5208             0.5279   

                          cv_accuracy_mean  test_f1  test_recall  \
Decision Tree Classifier            0.6804   0.1788       0.4113   
K-Nearest Neighbors                 0.5612   0.1551       0.3960   
Logistic Regression                 0.5278   0.1100       0.2656   
Support Vector Machine              0.5268   0.1080       0.2573   

                          test_precision  test_accuracy  
Decision Tree Classifier          0.1142         0.5794  
K-Nearest Neighbors               0.0965         0.5199  
Logistic Regression               0.0693         0.5216  
Support Vector Machine            0.0683         0.

#### Predicción con datos nuevos (sin clasificar)

In [35]:
df_nuevos = pd.read_csv('../data/raw/bank-additional-new.csv')
df_nuevos.head(5)

,age,job,marital,education,housing,loan,contact,month,day_of_week,duration,campaign,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y,contacted_before
0,56,housemaid,married,basic.4y,no,no,telephone,may,mon,261,1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,NaN
1,57,services,married,high.school,no,no,telephone,may,mon,149,1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,NaN
2,37,services,married,high.school,yes,no,telephone,may,mon,226,1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,NaN
3,40,admin.,married,basic.6y,no,no,telephone,may,mon,151,1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,NaN
4,56,services,married,high.school,no,yes,telephone,may,mon,307,1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,NaN


In [36]:
# Diagnosticar el problema con los nuevos datos
print("Información del conjunto de datos nuevos:")
print(f"Forma: {df_nuevos.shape}")
print("\nTipos de datos:")
print(df_nuevos.dtypes)
print("\nValores nulos:")
print(df_nuevos.isnull().sum())
print("\nColumnas categóricas en nuevos datos:")
print(df_nuevos.select_dtypes(include='object').columns.tolist())
print("\nColumnas numéricas en nuevos datos:")
print(df_nuevos.select_dtypes(exclude='object').columns.tolist())

Información del conjunto de datos nuevos:
Forma: (9, 20)

Tipos de datos:
age                   int64
job                  object
marital              object
education            object
housing              object
loan                 object
contact              object
month                object
day_of_week          object
duration              int64
campaign              int64
previous              int64
poutcome             object
emp_var_rate        float64
cons_price_idx      float64
cons_conf_idx       float64
euribor3m           float64
nr_employed         float64
y                    object
contacted_before    float64
dtype: object

Valores nulos:
age                 0
job                 0
marital             0
education           0
housing             0
loan                0
contact             0
month               0
day_of_week         0
duration            0
campaign            0
previous            0
poutcome            0
emp_var_rate        0
cons_price_idx      0
cons_c

In [37]:
# Comparar con los datos de entrenamiento originales
print("Comparación de columnas:")
print(f"Columnas en datos originales: {list(X.columns)}")
print(f"Columnas en datos nuevos: {list(df_nuevos.columns)}")

print("\nColumnas que están en nuevos pero no en originales:")
new_cols = set(df_nuevos.columns) - set(X.columns)
print(new_cols)

print("\nColumnas que están en originales pero no en nuevos:")
missing_cols = set(X.columns) - set(df_nuevos.columns)
print(missing_cols)

Comparación de columnas:
Columnas en datos originales: ['age', 'job', 'marital', 'education', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'emp_var_rate', 'cons_price_idx', 'cons_conf_idx', 'euribor3m', 'nr_employed']
Columnas en datos nuevos: ['age', 'job', 'marital', 'education', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'duration', 'campaign', 'previous', 'poutcome', 'emp_var_rate', 'cons_price_idx', 'cons_conf_idx', 'euribor3m', 'nr_employed', 'y', 'contacted_before']

Columnas que están en nuevos pero no en originales:
{'y', 'contacted_before'}

Columnas que están en originales pero no en nuevos:
{'pdays'}


In [38]:
# Se hace la predicción con los nuevos datos
# Primero, eliminar la columna objetivo si existe y preparar las características
X_new = df_nuevos.drop('y', axis=1) if 'y' in df_nuevos.columns else df_nuevos.copy()

# Asegurar que las columnas estén en el mismo orden que en el entrenamiento
X_new = X_new[X.columns]

# Manejar la columna contacted_before para que coincida con el formato de entrenamiento
# En entrenamiento: 'no', 'yes' (string)
# En nuevos datos: NaN -> necesita convertirse a 'no' (asumiendo que NaN significa no contactado)
X_new['contacted_before'] = X_new['contacted_before'].fillna('no')

# Convertir cualquier valor numérico a string si es necesario
if X_new['contacted_before'].dtype in ['float64', 'int64']:
    X_new['contacted_before'] = X_new['contacted_before'].map({0.0: 'no', 1.0: 'yes'}).fillna('no')

# Asegurar que contacted_before sea de tipo object como en entrenamiento
X_new['contacted_before'] = X_new['contacted_before'].astype('object')

# Transformar los nuevos datos usando el mismo preprocesador y predecir
try:
    x_new_prep = preprocessor_full.transform(X_new)
    
    y_new_pred = tree.predict(x_new_prep)
    print(f"\nPredicciones: {y_new_pred}")
    
    predictions_df = pd.DataFrame({
        'Cliente': range(1, len(y_new_pred) + 1),
        'Predicción_Numérica': y_new_pred,
        'Suscribirá': ['No' if pred == 0 else 'Sí' for pred in y_new_pred]
    })
    print("\nResultados detallados:")
    print(predictions_df.to_string(index=False))
    
    # Resumen de predicciones
    pred_counts = pd.Series(y_new_pred).value_counts()
    print("\nResumen de predicciones:")
    for pred_val, count in pred_counts.items():
        label = 'No realizará un depósito' if pred_val == 0 else 'Sí realizará un depósito'
        print(f"  {label}: {count} clientes ({count/len(y_new_pred)*100:.1f}%)")
    
except Exception as e:
    print(f"Error durante el preprocesamiento o predicción: {e}")
    print("Información adicional para depuración:")
    print(f"Tipos de datos en X_new:\n{X_new.dtypes}")

KeyError: "['pdays'] not in index"